### eBay API

In [12]:
import urllib
import os
import json

import coremltools 
from skafossdk import *
import turicreate as tc
import pandas as pd
from s3fs.core import S3FileSystem

In [3]:
s3 = S3FileSystem(anon=True)
json_path = "s3://mf.testing.bucket/Boots/ebay_boots.json"

df = pd.read_json(json_path)

In [23]:
#components = pd.DataFrame.from_dict(df.itemSummaries)

#components = df.itemSummaries.apply(json.loads).values.tolist()
components_df = pd.DataFrame.from_records(df.itemSummaries)

components_df.head()

,additionalImages,adultOnly,buyingOptions,categories,condition,conditionId,currentBidPrice,epid,image,itemAffiliateWebUrl,...,itemGroupType,itemHref,itemId,itemLocation,itemWebUrl,marketingPrice,price,seller,shippingOptions,title
0,[{'imageUrl': 'https://galleryplus.ebayimg.com...,False,[FIXED_PRICE],"[{'categoryId': '11498'}, {'categoryId': '1145...",New with box,1000,"{'value': '144.95', 'currency': 'USD'}",NaN,{'imageUrl': 'https://i.ebayimg.com/thumbs/ima...,https://rover.ebay.com/rover/1/711-53200-19255...,...,SELLER_DEFINED_VARIATIONS,https://api.ebay.com/buy/browse/v1/item/v1|112...,v1|112969366208|413347600515,"{'postalCode': '29575', 'country': 'US'}",https://www.ebay.com/itm/Red-Wing-Irish-Setter...,NaN,"{'value': '144.95', 'currency': 'USD'}","{'username': 'wildwestbootstore', 'feedbackPer...","[{'shippingCostType': 'FIXED', 'shippingCost':...","Red Wing Irish Setter Men's Ashby 6"" Aluminum ..."
1,[{'imageUrl': 'https://galleryplus.ebayimg.com...,False,[FIXED_PRICE],"[{'categoryId': '11498'}, {'categoryId': '1145...",New without box,1500,"{'value': '27.89', 'currency': 'USD'}",NaN,{'imageUrl': 'https://i.ebayimg.com/thumbs/ima...,https://rover.ebay.com/rover/1/711-53200-19255...,...,SELLER_DEFINED_VARIATIONS,https://api.ebay.com/buy/browse/v1/item/v1|271...,v1|271935356008|570772142390,{'country': 'US'},https://www.ebay.com/itm/Mens-Boots-Jungle-GI-...,NaN,"{'value': '27.89', 'currency': 'USD'}","{'username': 'abcpumc', 'feedbackPercentage': ...","[{'shippingCostType': 'FIXED', 'shippingCost':...",Men's Boots Jungle GI Type Black Tactical Comb...
2,[{'imageUrl': 'https://galleryplus.ebayimg.com...,False,[FIXED_PRICE],"[{'categoryId': '11498'}, {'categoryId': '1778...",New with box,1000,"{'value': '34.99', 'currency': 'USD'}",NaN,{'imageUrl': 'https://i.ebayimg.com/thumbs/ima...,https://rover.ebay.com/rover/1/711-53200-19255...,...,SELLER_DEFINED_VARIATIONS,https://api.ebay.com/buy/browse/v1/item/v1|372...,v1|372467315907|641272169972,"{'postalCode': '89103', 'country': 'US'}",https://www.ebay.com/itm/Original-SWAT-Mens-Ta...,"{'originalPrice': {'value': '119.99', 'currenc...","{'value': '34.99', 'currency': 'USD'}","{'username': 'botach', 'feedbackPercentage': '...","[{'shippingCostType': 'FIXED', 'shippingCost':...",Original SWAT Men's Tactical Duty Boots - Mult...
3,[{'imageUrl': 'https://galleryplus.ebayimg.com...,False,[FIXED_PRICE],"[{'categoryId': '53557'}, {'categoryId': '3034...",New without box,1500,"{'value': '17.09', 'currency': 'USD'}",NaN,{'imageUrl': 'https://i.ebayimg.com/thumbs/ima...,https://rover.ebay.com/rover/1/711-53200-19255...,...,SELLER_DEFINED_VARIATIONS,https://api.ebay.com/buy/browse/v1/item/v1|283...,v1|283085554574|583488755298,{'country': 'US'},https://www.ebay.com/itm/Womens-Low-Heels-Ankl...,"{'originalPrice': {'value': '17.99', 'currency...","{'value': '17.09', 'currency': 'USD'}","{'username': 'ochirchic', 'feedbackPercentage'...","[{'shippingCostType': 'FIXED', 'shippingCost':...",Womens Low Heels Ankle Boots Booties Round Toe...
4,[{'imageUrl': 'https://galleryplus.ebayimg.com...,False,[FIXED_PRICE],"[{'categoryId': '11498'}, {'categoryId': '1145...",New with box,1000,"{'value': '39.95', 'currency': 'USD'}",NaN,{'imageUrl': 'https://i.ebayimg.com/thumbs/ima...,https://rover.ebay.com/rover/1/711-53200-19255...,...,SELLER_DEFINED_VARIATIONS,https://api.ebay.com/buy/browse/v1/item/v1|362...,v1|362525699330|631678715798,"{'postalCode': '90013', 'country': 'US'}",https://www.ebay.com/itm/Magnum-6-Mens-Precisi...,"{'originalPrice': {'value': '140.00', 'currenc...","{'value': '39.95', 'currency': 'USD'}","{'username': 'altatac', 'feedbackPercentage': ...","[{'shippingCostType': 'FIXED', 'shippingCost':...","Magnum 6"" Mens Precision Ultra Lite II Waterpr..."


In [54]:
ebay_metadata = pd.DataFrame()
ebay_metadata['thumb_image'] = pd.DataFrame.from_records(components_df.image)['imageUrl']
ebay_metadata['price'] = pd.DataFrame.from_records(components_df.price)['value']
ebay_metadata['url'] = components_df.itemWebUrl
ebay_metadata['Description'] = components_df.title
ebay_metadata['path'] = ebay_metadata['thumb_image'].apply(lambda x: 'BootImages/' + x.split("/")[-2] + x.split("/")[-1])
ebay_metadata['Image Name'] = ebay_metadata['thumb_image'].apply(lambda x: x.split("/")[-2] + x.split("/")[-1])

In [53]:
for index, row in ebay_metadata.iterrows():
    urllib.request.urlretrieve(row['thumb_image'], row['path'])

In [55]:
ebay_metadata.head()

,thumb_image,price,url,Description,path,Image Name
0,https://i.ebayimg.com/thumbs/images/m/mn8bD8g-...,144.95,https://www.ebay.com/itm/Red-Wing-Irish-Setter...,"Red Wing Irish Setter Men's Ashby 6"" Aluminum ...",BootImages/mn8bD8g-P1o9SUbJraOfYzgs-l225.jpg,mn8bD8g-P1o9SUbJraOfYzgs-l225.jpg
1,https://i.ebayimg.com/thumbs/images/m/mubAOl9B...,27.89,https://www.ebay.com/itm/Mens-Boots-Jungle-GI-...,Men's Boots Jungle GI Type Black Tactical Comb...,BootImages/mubAOl9B7Q-mlzU5rjLg9HQs-l225.jpg,mubAOl9B7Q-mlzU5rjLg9HQs-l225.jpg
2,https://i.ebayimg.com/thumbs/images/m/m8skGCL-...,34.99,https://www.ebay.com/itm/Original-SWAT-Mens-Ta...,Original SWAT Men's Tactical Duty Boots - Mult...,BootImages/m8skGCL-RFkU-2-vbqDXkWgs-l225.jpg,m8skGCL-RFkU-2-vbqDXkWgs-l225.jpg
3,https://i.ebayimg.com/thumbs/images/m/mW7C3AsB...,17.09,https://www.ebay.com/itm/Womens-Low-Heels-Ankl...,Womens Low Heels Ankle Boots Booties Round Toe...,BootImages/mW7C3AsB96wPVjHGV9N8grgs-l225.jpg,mW7C3AsB96wPVjHGV9N8grgs-l225.jpg
4,https://i.ebayimg.com/thumbs/images/m/m_qHdL7E...,39.95,https://www.ebay.com/itm/Magnum-6-Mens-Precisi...,"Magnum 6"" Mens Precision Ultra Lite II Waterpr...",BootImages/m_qHdL7ELAehi_KrR1pXXDAs-l225.jpg,m_qHdL7ELAehi_KrR1pXXDAs-l225.jpg
